In [1]:
%cd ..
%pwd

/Users/brunostordeur/Documents/GitHub/End-to-End-Deep-Learning-Project


'/Users/brunostordeur/Documents/GitHub/End-to-End-Deep-Learning-Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path  
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_chkpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_chkpt_dir),
                            Path(config.tensorboard_root_log_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(
                root_dir = Path(config.root_dir),
                tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath =  Path(config.checkpoint_model_filepath)
                
            )
        return prepare_callbacks_config

In [29]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [32]:
class PrepareCallbacks:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
            )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_chkpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_chkpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_chkpt_callbacks
        ]

In [33]:
try:  
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    prepare_callbacks.get_chkpt_callbacks()
except Exception as e:
    raise e

[2023-07-20 17:05:11,079: INFO: common: yaml file: <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-07-20 17:05:11,081: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfuly]
[2023-07-20 17:05:11,081: INFO: common: created directory at: artifacts]
[2023-07-20 17:05:11,082: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-07-20 17:05:11,082: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
